In [1]:
from typing import Sequence

import clingo
import clingo.ast
from clingo.ast import ProgramBuilder
from clingox.backend import SymbolicBackend
from clingox.program import Program, ProgramObserver

In [2]:
program = """

head1(X,Y,Z) :- body1(X), body2(Y), body3(Z).

head2(X) :- body1(X), body2(X-1).

head3(X) :- body4(X).

"""

In [3]:
def get_parsed_program(program: str):
    nodes = []
    clingo.ast.parse_string(program, lambda stm: nodes.append(stm))
    return nodes

In [4]:
nodes = get_parsed_program(program)
nodes

[ast.Program(Location(begin=Position(filename='<string>', line=1, column=1), end=Position(filename='<string>', line=1, column=1)), 'base', []),
 ast.Rule(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=46)), ast.Literal(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=3, column=1), end=Position(filename='<string>', line=3, column=13)), 'head1', [ast.Variable(Location(begin=Position(filename='<string>', line=3, column=7), end=Position(filename='<string>', line=3, column=8)), 'X'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=9), end=Position(filename='<string>', line=3, column=10)), 'Y'), ast.Variable(Location(begin=Position(filename='<string>', line=3, column=11), end=Position(filename='<string>', line=3, column=12)), 'Z')], 0))), [ast.Literal(

Ground with symbols

In [5]:
class PartialGroundTransformer(clingo.ast.Transformer):
    pos = clingo.ast.Position('<string>', 0, 0)
    loc = clingo.ast.Location(pos, pos)

    def __init__(self, facts: Sequence[clingo.Symbol] = ()):
        self.grounded_rules = []
        self.facts = facts
        print('Available Facts: {', end='')
        if self.facts:
            print()
            print('.\n'.join(map(str, facts)), end='.\n')
        else:
            print(end=' ')
        print('}')

    def visit_Rule(self, rule: clingo.ast.AST):
        print('Partially grounding:', rule)
        ctl = clingo.Control()
        prg = Program()
        obs = ProgramObserver(prg)
        ctl.register_observer(obs)
        with SymbolicBackend(ctl.backend()) as sb:
            for fact in self.facts:
                sb.add_external(fact)
        with ProgramBuilder(ctl) as pb:
            pb.add(rule)
        ctl.ground([('base', ())])
        for grd_rule in prg.rules:
            head_atoms = grd_rule.head
            body_literals = grd_rule.body
            head_symbols = tuple(prg.output_atoms[head_atom] for head_atom in head_atoms)
            body_symbols = tuple(
                (int(body_literal < 0), prg.output_atoms[abs(body_literal)]) for body_literal in body_literals)
            print('Instance:', end=' ')
            if head_symbols:
                if grd_rule.choice:
                    print('{', ';'.join(map(str, head_symbols)), '}', end='')
                    # { H1;...;HN } :- B1,...,BN.
                    pass
                elif len(head_symbols) == 1:
                    head_symbol = head_symbols[0]
                    print(head_symbol, end='')
                    # H :- B1,...,BN.
                    head_ast = clingo.ast.Literal(PartialGroundTransformer.loc,
                                                  clingo.ast.Sign.NoSign,
                                                  clingo.ast.SymbolicAtom(
                                                      clingo.ast.SymbolicTerm(PartialGroundTransformer.loc,
                                                                              head_symbol)))
                else:
                    # H1 | ... | HN :- B1,...,BN.
                    print('|'.join(map(str, head_symbols)), end='')
                    pass
            else:
                head_ast = clingo.ast.BooleanConstant(False)

            body_ast = tuple(
                clingo.ast.Literal(PartialGroundTransformer.loc,
                                   sign,
                                   clingo.ast.SymbolicAtom(
                                       clingo.ast.SymbolicTerm(PartialGroundTransformer.loc,
                                                               body_atom)))
                for (sign, body_atom) in body_symbols
            )
            if body_ast:
                print(' :-', end=' ')
                print(', '.join(map(str, body_ast)), end='')
            print('.')
            self.grounded_rules.append(clingo.ast.Rule(PartialGroundTransformer.loc,
                                                       head_ast,
                                                       body_ast))

        return rule

In [6]:
pgt = PartialGroundTransformer([
    clingo.Function('body1', [clingo.Number(3)]),
    clingo.Function('body2', [clingo.Number(2)]),
    clingo.Function('body3', [clingo.Number(1)]),
    clingo.Function('body4', [clingo.Number(0)]),
    clingo.Function('body4', [clingo.Number(4)]),
])
clingo.ast.parse_string(program, lambda stm: pgt.visit(stm))

Available Facts: {
body1(3).
body2(2).
body3(1).
body4(0).
body4(4).
}
Partially grounding: head1(X,Y,Z) :- body1(X); body2(Y); body3(Z).
Instance: head1(3,2,1) :- body3(1), body2(2), body1(3).
Partially grounding: head2(X) :- body1(X); body2((X-1)).
Instance: head2(3) :- body2(2), body1(3).
Partially grounding: head3(X) :- body4(X).
Instance: head3(0) :- body4(0).
Instance: head3(4) :- body4(4).


In [7]:
print('\n'.join(map(str, pgt.grounded_rules)))

head1(3,2,1) :- body3(1); body2(2); body1(3).
head2(3) :- body2(2); body1(3).
head3(0) :- body4(0).
head3(4) :- body4(4).


In [8]:
new_ctl = clingo.Control()
new_ctl.configuration.solve.models = 0

In [9]:
with ProgramBuilder(new_ctl) as pb:
    for gr in pgt.grounded_rules:
        pb.add(gr)
with SymbolicBackend(new_ctl.backend()) as sb:
    for f in pgt.facts:
        sb.add_rule(head=(f,))

In [10]:
new_ctl.ground([('base', [])])

In [11]:
with new_ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = sorted(model.symbols(shown=True))
        print("Answer {}:".format(model.number), end=' ')
        print("{",
              '\n'.join(map(str, symbols)), "}", sep='\n')
        models.append(symbols)
    solve_result = solve_handle.get()
    print(solve_result, end='')
    if models:
        print(" {}{}".format(len(models), '' if solve_result.exhausted else '+'))

Answer 1: {
body1(3)
body2(2)
body3(1)
body4(0)
body4(4)
head2(3)
head3(0)
head3(4)
head1(3,2,1)
}
SAT 1
